## 1. Base dataframe

In [260]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

output_directory='/content/drive/MyDrive/MASTER DATA SCIENCE/M2 internship/epigene labs challenge'

data_dir='/content/drive/MyDrive/MASTER DATA SCIENCE/M2 internship/epigene labs challenge/dataset'

all_paths=os.listdir(data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [243]:
##============================== get columns with more variability from df 1 ====================##

'''   create base dataframe with columns with more variability accross files     '''

df_1=pd.read_csv(os.path.join(data_dir,all_paths[0]))

variable_columns=[]

for column in df_1.columns[~df_1.columns.str.contains('characteristics')]:

  if df_1[column].dtype=='O' and df_1[column].nunique()>1:
    variable_columns.append(column)

  elif df_1[column].dtype=='int64' and df_1[column].std()>0:
    variable_columns.append(column)

variable_columns.remove('last_update_date')
print('number of variable columns:', len(variable_columns))
print(variable_columns)

number of variable columns: 5
['description', 'geo_accession', 'source_name_ch1', 'supplementary_file', 'title']


In [244]:
##============================== concatenate all dfs ====================##

''' keep columns with more variability from df 1, and all characteristics columns '''

raw_df=pd.DataFrame()
total_rows=0
for file in all_paths:

  temp=pd.read_csv(os.path.join(data_dir,file))
  temp['file_name']=file

  total_rows+=temp.shape[0]
  total_columns=variable_columns+list(temp.columns[temp.columns.str.contains('characteristics')])
  total_columns.append('file_name')

  raw_df=pd.concat([raw_df,temp[total_columns]])

## 2. Gender feature

In [245]:
#======= get gender from source_name_ch1 column =============================##

df=pd.DataFrame()
df['file_name']=raw_df['file_name']
df['source_name_ch1']=raw_df['source_name_ch1']

key_words=['ovar']

df['sex']=raw_df['source_name_ch1'].str.lower().str.contains('|'.join(key_words)).apply(
    lambda x: 'female' if x else np.nan
)

In [246]:
##========= get gender from  keywords search =========##
gender_columns = [col for col in raw_df.columns if 'gender' in col.lower()]

df['sex'].fillna(raw_df[gender_columns].bfill(axis=1).iloc[:, 0],inplace=True)

<ipython-input-246-f34fcb56a735>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sex'].fillna(raw_df[gender_columns].bfill(axis=1).iloc[:, 0],inplace=True)


## 3. Age feature

In [247]:
def detect_columns_with_exact_word(data_dir, file_paths, list_words):
    """
    Detect columns across multiple files that contain a specific word within text, or column name.

    Args:
        data_dir (str): Directory containing the files.
        file_paths (list): List of file paths to process.
        word (str): The word to search for within text and column names.

    Returns:
        dict: Dictionary with keys being files, and values being column names containing the word.
    """
    results = []

    for word in list_words:

      word_pattern = rf'\b{word}\b'

      for file in file_paths:
        df = pd.read_csv(os.path.join(data_dir, file))

        ## check column names
        #columns=df.iloc[:,df.columns.str.lower().str.contains(word)].columns

        for column in df.columns:
          if word_pattern in column.lower() and (column not in results):
            results.append(column)

      for file in file_paths:
        df = pd.read_csv(os.path.join(data_dir, file))

        for column in df.columns:

          ## check column content
          if df[column].astype(str).str.contains(word_pattern, case=False, na=False, regex=True).any():
              results.append(column)

    return results

In [248]:
columns_with_age = detect_columns_with_exact_word(data_dir, all_paths, ["age"])
df['age']=raw_df[raw_df.columns.intersection(columns_with_age)].bfill(axis=1).iloc[:, 0]

df['age'] = df['age'].str.extract(r'(\d+)')  # Extract numbers
df['age'] = pd.to_numeric(df['age'], errors='coerce')  # Convert to numeric, keeping NaNs
df['age']=df['age'].astype('Int64')

## 4. Cancer stage feature

In [249]:
##================= stage column ======================##
columns_with_stage = detect_columns_with_exact_word(data_dir, all_paths, ["stage"])
stage=raw_df[raw_df.columns.intersection(columns_with_stage)].bfill(axis=1).iloc[:, 0]
stage=pd.DataFrame(stage)

In [250]:
def normalize_stage(value):

    if value is None:
        return np.nan
    value = str(value).lower()  # Convert to string and lowercase for consistency
    if ('1' in value) or ('i' in value) or ('early' in value):
        return 'I'
    elif ('2' in value) or ('ii' in value):
        return 'II'
    elif ('3' in value) or ('iii' in value):
        return 'III'
    elif ('4' in value) or ('iv' in value) or ('advanced' in value):
        return 'IV'
    #return np.nan

# Apply the normalization function to the column
stage['normalized']=stage[stage.columns[0]].apply(normalize_stage)

df['stage']=stage['normalized']

## 5. Cancer grade feature

In [251]:
##================= grade column ======================##
columns_with_grade = detect_columns_with_exact_word(data_dir, all_paths, ["grade"])
grade=raw_df[raw_df.columns.intersection(columns_with_grade)].bfill(axis=1).iloc[:, 0]
grade=pd.DataFrame(grade)


def normalize_grade(value):

    if value is None:
        return np.nan
    value = str(value).lower()  # Convert to string and lowercase for consistency

    if ('1' in value):
        return 'I'
    elif ('2' in value):
        return 'II'
    elif ('3' in value):
        return 'III'
    elif ('4' in value):
        return 'IV'
    return np.nan

# Apply the normalization function to the column
grade['normalized']=grade[grade.columns[0]].apply(normalize_grade)

df['grade']=grade['normalized']

## 6. Cancer type

In [252]:
##================= cancer_type column ======================##
columns_with_cancer_type = detect_columns_with_exact_word(data_dir, all_paths, ["ovar","breast"])
cancer_type=raw_df[raw_df.columns.intersection(columns_with_cancer_type)].bfill(axis=1).iloc[:, 0]
cancer_type=pd.DataFrame(cancer_type)

df['cancer_type']=cancer_type[cancer_type.columns[0]].apply(
    lambda x: 'ovarian' if 'ovar' in str(x).lower()
              else 'breast' if 'breast' in str(x).lower()
              else 'unaffected'
                                    )

## 7. Sample type

In [253]:
columns_with_sample_type = detect_columns_with_exact_word(data_dir, all_paths, ['type','tumor','tissue','primary','normal'])

unique_types=['primary','normal']

df['sample_type'] = (
    raw_df[raw_df.columns.intersection(columns_with_sample_type)]
    .apply(lambda row: ' '.join(row.dropna().astype(str).str.lower()), axis=1)
    .apply(lambda x: next((kw for kw in unique_types if kw in x), np.nan))
)


## 8. Cancer subtype

In [254]:
subtype_list=["type","tumor","tissue","serous", "endometrioid",
"mucinous", "clear cell", "ductal", "lobular", "apocrine"]

columns_with_cancer_subtype = detect_columns_with_exact_word(data_dir, all_paths, subtype_list) ##type, tumor, tissue

unique_subtypes=raw_df['characteristics_ch1.0.Tissue'].dropna().apply(lambda x: x.split(',')[1]).apply(lambda x: x.split(': ')[-1]).drop_duplicates().reset_index(drop=True)

unique_subtypes=unique_subtypes.values

df['tumor_subtype'] = (
    raw_df[raw_df.columns.intersection(columns_with_cancer_subtype)]
    .apply(lambda row: ' '.join(row.dropna().astype(str).str.lower()), axis=1)
    .apply(lambda x: next((kw for kw in unique_subtypes if kw in x), np.nan))
)


## Final dataframe

In [258]:
main_columns=['sex', 'age', 'stage', 'grade','cancer_type', 'tumor_subtype', 'sample_type']
df=df[main_columns]

# Add a temporary column to count NaN values in each row
df['nan_count'] = df.isna().sum(axis=1)

# Sort the DataFrame by the `nan_count` column in ascending order
df = df.sort_values(by='nan_count', ascending=True).drop(columns=['nan_count'])

# Reset the index for cleaner output (optional)
df.reset_index(drop=True, inplace=True)

# Step 1: Identify the first occurrence of each unique combination
df['is_first_occurrence'] = ~df.duplicated(subset=main_columns, keep='first')

# Step 2: Create sequential IDs for the first occurrences only
id_counter = iter(range(1, df['is_first_occurrence'].sum() + 1))
df['patient_id'] = df['is_first_occurrence'].apply(lambda x: next(id_counter) if x else np.nan)

# Step 3: Drop the helper column (optional)
df = df.drop(columns=['is_first_occurrence'])

df=df[['patient_id','sex', 'age', 'stage', 'grade','cancer_type', 'tumor_subtype', 'sample_type']]

df.sort_values(by='patient_id')

,patient_id,sex,age,stage,grade,cancer_type,tumor_subtype,sample_type
0,1.0,female,54,I,NaN,ovarian,serous,NaN
1,2.0,female,70,I,NaN,ovarian,serous,NaN
2,3.0,female,51,I,NaN,ovarian,serous,NaN
3,4.0,female,68,I,NaN,ovarian,serous,NaN
4,5.0,female,62,I,NaN,ovarian,serous,NaN
...,...,...,...,...,...,...,...,...
1063,NaN,NaN,<NA>,None,NaN,breast,NaN,NaN
1064,NaN,NaN,<NA>,None,NaN,breast,NaN,NaN
1065,NaN,NaN,<NA>,None,NaN,breast,NaN,NaN
1066,NaN,NaN,<NA>,None,NaN,breast,NaN,NaN


In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     75 non-null     float64
 1   sex            479 non-null    object 
 2   age            54 non-null     Int64  
 3   stage          959 non-null    object 
 4   grade          462 non-null    object 
 5   cancer_type    1068 non-null   object 
 6   tumor_subtype  270 non-null    object 
 7   sample_type    490 non-null    object 
dtypes: Int64(1), float64(1), object(6)
memory usage: 67.9+ KB


In [261]:
df.to_csv(f'{output_directory}clinical_meta_data.csv',index=False)